In [ ]:
#importing prerequisite libraries and models 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from matplotlib.colors import ListedColormap
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from matplotlib import pyplot as plt
from google.colab import drive
from sklearn import tree
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2




In [ ]:
#reading data_1 csv file
dataset=pd.read_csv("/content/drive/MyDrive/data_1.csv")   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#separating independent and target variables
X=dataset.iloc[:,:-1]     
Y=dataset.iloc[:,-1]
Y=pd.DataFrame(Y)
X.shape

(1450, 21)

In [ ]:
#splitting the data into training and testing data in the ratio of 80:20
X_train1,X_test1,Y_train1,Y_test1= train_test_split(X,Y,test_size=0.20,random_state=0)    

In [ ]:
#training the Decision Tree model
clf=DecisionTreeClassifier(random_state=0)
clf.fit(X_train1,Y_train1)
Y_test1_predictions=clf.predict(X_test1)
Y_test1_predictions=pd.DataFrame(Y_test1_predictions)

In [ ]:
#reading data_2 csv file
dataset2=pd.read_csv("/content/drive/MyDrive/data_2.csv") 

In [ ]:
#separating independent and target variables on data_2
X2=dataset2.iloc[:,:-1]     
Y2=dataset2.iloc[:,-1]
Y2=pd.DataFrame(Y2)

In [ ]:
#splitting the data into training and testing data in the ratio of 80:20 of data_2
X_train2,X_test2,Y_train2,Y_test2= train_test_split(X2,Y2,test_size=0.20,random_state=0)  
X_train2=X_train2.reset_index(drop=True)
X_train2

baseline value  accelerations  ...  histogram_variance  histogram_tendency
0               138          0.011  ...                  21                   1
1               110          0.004  ...                  36                  -1
2               138          0.006  ...                  22                   1
3               143          0.000  ...                   1                   0
4               134          0.009  ...                  38                   0
..              ...            ...  ...                 ...                 ...
535             132          0.000  ...                   9                   1
536             125          0.000  ...                   9                   1
537             143          0.000  ...                   1                   0
538             138          0.000  ...                 113                   1
539             132          0.000  ...                  86                  -1

[540 rows x 21 columns]

In [ ]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold

node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
while len(stack) > 0:
    # `pop` ensures each node is only visited once
    node_id, depth = stack.pop()
    node_depth[node_id] = depth

    # If the left and right child of a node is not the same we have a split
    # node
    is_split_node = children_left[node_id] != children_right[node_id]
    # If a split node, append left and right children and depth to `stack`
    # so we can loop through them
    if is_split_node:
        stack.append((children_left[node_id], depth + 1))
        stack.append((children_right[node_id], depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has {n} nodes and has "
      "the following tree structure:\n".format(n=n_nodes))
for i in range(n_nodes):
    if is_leaves[i]:
        print("{space}node={node} is a leaf node.".format(
            space=node_depth[i] * "\t", node=i))
    else:
        print("{space}node={node} is a split node: "
              "go to node {left} if X[:, {feature}] <= {threshold} "
              "else to node {right}.".format(
                  space=node_depth[i] * "\t",
                  node=i,
                  left=children_left[i],
                  feature=feature[i],
                  threshold=threshold[i],
                  right=children_right[i]))

The binary tree structure has 105 nodes and has the following tree structure:

node=0 is a split node: go to node 1 if X[:, 7] <= 59.5 else to node 80.
	node=1 is a split node: go to node 2 if X[:, 7] <= 52.5 else to node 45.
		node=2 is a split node: go to node 3 if X[:, 17] <= 107.5 else to node 6.
			node=3 is a split node: go to node 4 if X[:, 14] <= 3.5 else to node 5.
				node=4 is a leaf node.
				node=5 is a leaf node.
			node=6 is a split node: go to node 7 if X[:, 16] <= 158.5 else to node 30.
				node=7 is a split node: go to node 8 if X[:, 14] <= 15.0 else to node 29.
					node=8 is a split node: go to node 9 if X[:, 2] <= 0.1250000037252903 else to node 24.
						node=9 is a split node: go to node 10 if X[:, 15] <= 3.5 else to node 21.
							node=10 is a split node: go to node 11 if X[:, 6] <= 0.001500000071246177 else to node 18.
								node=11 is a split node: go to node 12 if X[:, 17] <= 150.5 else to node 13.
									node=12 is a leaf node.
									node=13 is a spl

In [ ]:
plt.figure(figsize=(20,25))
attributes = dataset.columns
labels = ['1','2','3']
tree.plot_tree(clf,feature_names=attributes,class_names=labels,filled=True)
plt.show()

In [ ]:
node_indicator = clf.decision_path(X_train1)
leaf_id = clf.apply(X_train1)

sampleid = np.arange(0,540)
for sample_id in sampleid:
    # obtain ids of the nodes `sample_id` goes through, i.e., row `sample_id`
    node_index = node_indicator.indices[node_indicator.indptr[sample_id]:node_indicator.indptr[sample_id + 1]]

    print('Rules used to predict sample {id}:\n'.format(id=sample_id))
    for node_id in node_index:
        # continue to the next node if it is a leaf node
        if leaf_id[sample_id] == node_id:
           continue

        # check if value of the split feature for sample 0 is below threshold
        if (X_train1.iloc[sample_id, feature[node_id]] <= threshold[node_id]):
             threshold_sign = "<="
        else:
              threshold_sign = ">"

        print("decision node {node} : (X_train1[{sample}, {feature}] = {value}) "
          "{inequality} {threshold})".format(
              node=node_id,
              sample=sample_id,
              feature=feature[node_id],
              value=X_train1.iloc[sample_id, feature[node_id]],
              inequality=threshold_sign,
              threshold=threshold[node_id]))

Streaming output truncated to the last 5000 lines.
decision node 83 : (X_train1[58, 2] = 0.004) <= 0.02650000061839819)
decision node 84 : (X_train1[58, 1] = 0.0) <= 0.0005000000237487257)
decision node 85 : (X_train1[58, 11] = 12) <= 79.5)
Rules used to predict sample 59:

decision node 0 : (X_train1[59, 7] = 51) <= 59.5)
decision node 1 : (X_train1[59, 7] = 51) <= 52.5)
decision node 2 : (X_train1[59, 17] = 137) > 107.5)
decision node 6 : (X_train1[59, 16] = 147) <= 158.5)
decision node 7 : (X_train1[59, 14] = 12) <= 15.0)
decision node 8 : (X_train1[59, 2] = 0.003) <= 0.1250000037252903)
decision node 9 : (X_train1[59, 15] = 4) > 3.5)
decision node 21 : (X_train1[59, 12] = 52) > 51.5)
Rules used to predict sample 60:

decision node 0 : (X_train1[60, 7] = 63) > 59.5)
decision node 80 : (X_train1[60, 9] = 36) <= 68.5)
decision node 81 : (X_train1[60, 7] = 63) <= 79.5)
decision node 82 : (X_train1[60, 1] = 0.0) <= 0.0025000000605359674)
decision node 83 : (X_train1[60, 2] = 0.0) <= 0.0

In [ ]:
node_indicator = clf.decision_path(X_train2)
leaf_id = clf.apply(X_train2)

sampleid = np.arange(0,540)
for sample_id in sampleid:
    # obtain ids of the nodes `sample_id` goes through, i.e., row `sample_id`
    node_index = node_indicator.indices[node_indicator.indptr[sample_id]:node_indicator.indptr[sample_id + 1]]

    print('Rules used to predict sample {id}:\n'.format(id=sample_id))
    for node_id in node_index:
        # continue to the next node if it is a leaf node
        if leaf_id[sample_id] == node_id:
           continue

        # check if value of the split feature for sample 0 is below threshold
        if (X_train2.iloc[sample_id, feature[node_id]] <= threshold[node_id]):
             threshold_sign = "<="
        else:
              threshold_sign = ">"

        print("decision node {node} : (X_train2[{sample}, {feature}] = {value}) "
          "{inequality} {threshold})".format(
              node=node_id,
              sample=sample_id,
              feature=feature[node_id],
              value=X_train2.iloc[sample_id, feature[node_id]],
              inequality=threshold_sign,
              threshold=threshold[node_id]))

Rules used to predict sample 0:

decision node 0 : (X_train2[0, 7] = 57) <= 59.5)
decision node 1 : (X_train2[0, 7] = 57) > 52.5)
decision node 45 : (X_train2[0, 13] = 183) <= 189.0)
decision node 46 : (X_train2[0, 12] = 135) <= 139.5)
decision node 47 : (X_train2[0, 2] = 0.0) <= 0.019499999471008778)
decision node 48 : (X_train2[0, 9] = 0) <= 52.0)
decision node 49 : (X_train2[0, 2] = 0.0) <= 0.00849999999627471)
decision node 50 : (X_train2[0, 10] = 3.7) <= 16.0)
Rules used to predict sample 1:

decision node 0 : (X_train2[1, 7] = 64) > 59.5)
decision node 80 : (X_train2[1, 9] = 0) <= 68.5)
decision node 81 : (X_train2[1, 7] = 64) <= 79.5)
decision node 82 : (X_train2[1, 1] = 0.004) > 0.0025000000605359674)
decision node 100 : (X_train2[1, 11] = 176) > 27.5)
Rules used to predict sample 2:

decision node 0 : (X_train2[2, 7] = 49) <= 59.5)
decision node 1 : (X_train2[2, 7] = 49) <= 52.5)
decision node 2 : (X_train2[2, 17] = 140) > 107.5)
decision node 6 : (X_train2[2, 16] = 142) <= 15